In [ ]:
import pandas as pd
import random
import heapq
from collections import deque, defaultdict

class Node:
    def __init__(self, name, mu, servers):
        self.name = name
        self.mu = mu # tasa de servicio de cada servidor
        self.servers = servers # número de servidores
        self.busy = 0 # servidores ocupados
        self.queue = deque() # cola de clientes (pid, tiempo_de_llegada_a_cola)
        # estadísticas
        self.area_q = 0.0 # integral de longitud de cola
        self.area_busy = 0.0 # integral de servidores ocupados
        self.last_event_time = 0.0
        self.num_served = 0 # número de clientes servidos

class Event:
    def __init__(self, time, kind, node_name, pid, external=False):
        self.time = time # instante del evento
        self.kind = kind # 'arrival' o 'departure'
        self.node_name = node_name # nodo donde ocurre
        self.pid = pid # identificador del cliente
        self.external = external # si es llegada externa
    def __lt__(self, other):
        return self.time < other.time

def simulate(T: float, lambda1: float, lambda2: float, nodes: dict[str, Node]
        ) -> dict[str, dict[str, float]]:
    """
    Args:
        T: Tiempo de simulación (horas).
        lambda1: Tasa de llegada externa a 'reg1' (pacientes/hora).
        lambda2: Tasa de llegada externa a 'reg2' (pacientes/hora).
        nodes: Diccionario <nombre, Node>
    
    Returns:
        metrics: Diccionario <nombre_nodo, dict<nombre_metrica, float>>
            Explicación de las métricas:
            - utilización: Proporción de tiempo ocupado por los servidores.
            - Lq: Longitud promedio de la cola (número de clientes en cola).
            - Wq: Tiempo promedio en cola (horas).
            - throughput: Rendimiento (número de clientes atendidos por hora).
    """
    # lista de eventos
    future_events = []
    t = 0.0
    pid_counter = 0

    # estadísticas de espera por nodo
    waits = defaultdict(list)

    # Programar primeras llegadas externas

    # lambda 1 y lambda 2 son tasas de llegada por hora
    # y el tiempo de llegada es exponencial con parámetro lambda
    # (1/lambda es el tiempo promedio entre llegadas)
    # Por lo tanto, la llegada de un cliente es un evento que ocurre a
    # t + expovariate(lambda) horas
    heapq.heappush(future_events, Event(random.expovariate(lambda1), 'arrival', 'reg1', pid_counter, True))
    pid_counter += 1
    heapq.heappush(future_events, Event(random.expovariate(lambda2), 'arrival', 'reg2', pid_counter, True))
    pid_counter += 1

    while future_events:
        event = heapq.heappop(future_events)
        if event.time > T:
            break
        # avanzamos el reloj
        prev_t = t
        t = event.time
        # actualizamos áreas bajo Q y busy para cada nodo
        for nd in nodes.values():
            dt = t - nd.last_event_time
            nd.area_q += len(nd.queue) * dt
            nd.area_busy += nd.busy * dt
            nd.last_event_time = t

        node = nodes[event.node_name]

        if event.kind == 'arrival':
            # si llegó externamente, programar siguiente llegada externa
            if event.external:
                if event.node_name == 'reg1':
                    heapq.heappush(future_events, Event(t + random.expovariate(lambda1),
                                           'arrival', 'reg1', pid_counter, True))
                else:
                    heapq.heappush(future_events, Event(t + random.expovariate(lambda2),
                                           'arrival', 'reg2', pid_counter, True))
                pid_counter += 1

            # llegada al nodo: si hay servidor libre se atiende enseguida
            if node.busy < node.servers:
                node.busy += 1
                waits[node.name].append(0.0)
                # programo la salida
                dt = random.expovariate(node.mu)
                heapq.heappush(future_events, Event(t + dt, 'departure', node.name, event.pid))
            else:
                # se pone en cola
                node.queue.append((event.pid, t))

        else: # departure
            node.num_served += 1
            node.busy -= 1
            # si hay cola, empieza el siguiente
            if node.queue:
                pid_q, t_arr_q = node.queue.popleft()
                wait_time = t - t_arr_q
                waits[node.name].append(wait_time)
                node.busy += 1
                dt = random.expovariate(node.mu)
                heapq.heappush(future_events, Event(t + dt, 'departure', node.name, pid_q))

            # determinar a dónde va el cliente
            if event.node_name == 'reg1':
                next_node = 'exam1'
            elif event.node_name == 'reg2':
                next_node = 'exam2'
            elif event.node_name.startswith('exam'):
                # aleatorio 50/50
                next_node = 'consult1' if random.random() < 0.5 else 'consult2'
            elif event.node_name.startswith('consult'):
                next_node = 'pharma1' if event.node_name == 'consult1' else 'pharma2'
            else:
                # en farmacia sale del sistema
                continue

            # llegada instantánea al siguiente nodo
            heapq.heappush(future_events, Event(t, 'arrival', next_node, event.pid, False))

    # métricas de desempeño
    metrics = {}
    for name, nd in nodes.items():
        Lq = nd.area_q / T
        rho = nd.area_busy / T / nd.servers
        Wq = (sum(waits[name]) / len(waits[name])) if waits[name] else 0.0
        throughput = nd.num_served / T
        metrics[name] = {
            'utilization': rho,
            'Lq': Lq,
            'Wq': Wq,
            'throughput': throughput
        }
    return metrics

if __name__ == '__main__':
    NUM_RUNS = 4 # número de corridas
    for i in range(NUM_RUNS):
        print(f"Corrida {i+1}/{NUM_RUNS}")
        # Semilla aleatoria basada en i y un número aleatorio
        rand_seed = random.randint(i*1000, (i+1)*1000)
        random.seed(rand_seed)
        print(f"Semilla aleatoria: {rand_seed}")

        # Definir nodos: name, mu, servers
        nodes = {
            'reg1': Node('reg1', mu=10.0, servers=1),
            'reg2': Node('reg2', mu=8.0, servers=1),
            'exam1': Node('exam1', mu=6.0, servers=1),
            'exam2': Node('exam2', mu=7.0, servers=1),
            'consult1':Node('consult1', mu=4.0, servers=2),
            'consult2':Node('consult2', mu=5.0, servers=2),
            'pharma1': Node('pharma1', mu=12.0, servers=1),
            'pharma2': Node('pharma2', mu=10.0, servers=1),
        }

        # Parámetros de llegada externa (pacientes/hora)
        lambda1 = 5.0 # a reg1
        lambda2 = 3.0 # a reg2

        T = 500.0 # horas de simulación

        results = simulate(T, lambda1, lambda2, nodes)

        # Mostrar resultados
        for name, m in results.items():
            print(f"{name:8s}: utilización={m['utilization']:.3f}, "
                  f"longitud de cola={m['Lq']:.3f}, "
                  f"tiempo en cola={m['Wq']:.3f}, "
                  f"rendimiento={m['throughput']:.3f}")
        
        print("\n"*3, "-" * 50)


Corrida 1/4
Semilla aleatoria: 84
reg1    : utilización=0.501, longitud de cola=0.491, tiempo en cola=0.100, rendimiento=4.904
reg2    : utilización=0.377, longitud de cola=0.249, tiempo en cola=0.083, rendimiento=2.990
exam1   : utilización=0.810, longitud de cola=3.548, tiempo en cola=0.720, rendimiento=4.866
exam2   : utilización=0.454, longitud de cola=0.399, tiempo en cola=0.134, rendimiento=2.990
consult1: utilización=0.485, longitud de cola=0.293, tiempo en cola=0.075, rendimiento=3.922
consult2: utilización=0.387, longitud de cola=0.141, tiempo en cola=0.036, rendimiento=3.932
pharma1 : utilización=0.322, longitud de cola=0.165, tiempo en cola=0.042, rendimiento=3.922
pharma2 : utilización=0.399, longitud de cola=0.265, tiempo en cola=0.067, rendimiento=3.932



 --------------------------------------------------
Corrida 2/4
Semilla aleatoria: 1942
reg1    : utilización=0.504, longitud de cola=0.513, tiempo en cola=0.103, rendimiento=4.998
reg2    : utilización=0.379, longitud 